In [2]:
import os
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import tkinter as tk

In [3]:
load_dotenv()

True

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
base_url = os.getenv("BASE_URL")

In [6]:
# Create the Alpaca API object
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url,
    api_version="v2"
)

In [7]:
# Define the stock tickers
tickers = ['PANW', 'LRCX', 'AAPL', 'AMZN', 'MSFT', 'TSLA', 'CAT', 'DE']

In [8]:
# Define the window periods for the EMAs
short_window = 13
middle_window = 20
long_window = 100

In [12]:
# Retrieve historical price data
barset = api.get_bars(tickers, '1Day', limit=long_window)

In [ ]:
# Calculate EMAs for each stock
ema_data = {}
for ticker in tickers:
    close_prices = [bar.c for bar in barset[ticker]]
    ema_short = np.convolve(close_prices, np.ones(short_window) / short_window, mode='valid')
    ema_middle = np.convolve(close_prices, np.ones(middle_window) / middle_window, mode='valid')
    ema_long = np.convolve(close_prices, np.ones(long_window) / long_window, mode='valid')
    ema_data[ticker] = {'short': ema_short, 'middle': ema_middle, 'long': ema_long}

In [ ]:
# Trading strategy
def execute_trades():
    for ticker in tickers:
        current_price = barset[ticker][-1].c
        ema_short = ema_data[ticker]['short'][-1]
        ema_middle = ema_data[ticker]['middle'][-1]
        ema_long = ema_data[ticker]['long'][-1]

        if ema_short > ema_middle and ema_middle > ema_long:
            # Place buy order
            api.submit_order(
                symbol=ticker,
                qty=1,  # Adjust quantity as desired
                side='buy',
                type='market',
                time_in_force='gtc'
            )
            log_text.insert(tk.END, f"Buy order placed for {ticker} at {current_price}\n")
        elif ema_short < ema_middle and ema_middle < ema_long:
            # Place sell order
            api.submit_order(
                symbol=ticker,
                qty=1,  # Adjust quantity as desired
                side='sell',
                type='market',
                time_in_force='gtc'
            )
            log_text.insert(tk.END, f"Sell order placed for {ticker} at {current_price}\n")

In [ ]:
# GUI interface using Tkinter
root = tk.Tk()
root.title("Trading Bot Interface")

In [ ]:
# Log text area
log_text = tk.Text(root, height=10, width=50)
log_text.pack()

In [ ]:
# Execute trades button
execute_button = tk.Button(root, text="Execute Trades", command=execute_trades)
execute_button.pack()

root.mainloop()